# LA Wildfire Prediction: Random Forest Model

In [21]:


import pandas as pd
import numpy as np
import os
import joblib
import time
from datetime import datetime
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# For displaying plots in the notebook
%matplotlib inline


## Data Loading
The following function loads the engineered wildfire dataset with all the features created in the previous notebook.


In [23]:
def load_engineered_data(file_path):
   
    print(f"Loading engineered data from {file_path}...")
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    
    # Print the head of loaded data
    print("\nLoaded Engineered Data Head:")
    print(df.head())
    
    return df


## Feature Selection
Select relevant features for the model from the engineered dataset.


In [25]:
def select_features(df):
  
    print("Selecting features...")
    
    # Weather features
    weather_features = ['AWND', 'PRCP', 'TMAX', 'TMIN']
    
    # Rolling features
    rolling_features = []
    for col in df.columns:
        if any(suffix in col for suffix in ['_7D', '_3D', '_14D', '_prev']):
            rolling_features.append(col)
    
    # Dryness features
    dryness_features = []
    for col in ['is_dry', 'dry_streak', 'days_since_rain', 'drought_index']:
        if col in df.columns:
            dryness_features.append(col)
    
    # Satellite features
    satellite_features = []
    if 'LST_Day_C' in df.columns:
        satellite_features.append('LST_Day_C')
    
    # Time features
    time_features = ['month', 'day_of_year', 'day_of_week', 'is_weekend']
    if 'season_Spring' in df.columns:
        time_features.extend(['season_Spring', 'season_Summer', 'season_Winter'])
    
    # Fire spread potential
    special_features = []
    if 'fire_spread_potential' in df.columns:
        special_features.append('fire_spread_potential')
    
    # Combine all features
    all_features = (weather_features + rolling_features + dryness_features +
                   satellite_features + time_features + special_features)
    
    # Filter to include only features that exist in the dataframe
    selected_features = [f for f in all_features if f in df.columns]
    
    print(f"Selected {len(selected_features)} features: {selected_features}")
    
    # Create X (features) and y (target)
    X = df[selected_features]
    y = df['Fire_Occurred']  # Use Fire_Occurred as the target variable
    
    # Print the head of selected features
    print("\nSelected Features Head:")
    print(X.head())
    
    return X, y, selected_features


## Class Imbalance Handling
Handle class imbalance in the dataset using class weights to give higher importance to the minority class.


In [27]:
def handle_class_imbalance(X, y, method='class_weight'):
  
    print("Handling class imbalance...")
    
    # Check class distribution
    class_counts = y.value_counts()
    print(f"Class distribution before balancing: {class_counts}")
    
    # Calculate class imbalance ratio
    if len(class_counts) > 1:
        imbalance_ratio = class_counts.min() / class_counts.max()
        print(f"Class imbalance ratio: {imbalance_ratio:.4f}")
    
    # For large datasets, using class weights is more efficient than resampling
    if method == 'class_weight' or X.shape[0] > 100000:
        print("Using class weights instead of resampling due to large dataset size")
        # Calculate class weights inversely proportional to class frequencies
        class_weight = {0: 1.0,
                       1: class_counts[0] / class_counts[1] if 1 in class_counts and class_counts[1] > 0 else 1.0}
        print(f"Class weights: {class_weight}")
        return X, y, class_weight
    
    # If no resampling is needed or possible
    else:
        print("No resampling applied")
        return X, y, None


## PCA Dimensionality Reduction
Apply Principal Component Analysis (PCA) to reduce the dimensionality of the feature space while retaining most of the variance. This helps improve model performance and reduce overfitting.



In [29]:
def apply_pca(X, explained_variance=0.95):
   
    print(f"\nApplying PCA to retain {explained_variance*100:.1f}% variance...")
    
    # 1. Create PCA model
    pca = PCA(n_components=explained_variance, random_state=42)
    
    # 2. Fit and transform the data
    X_pca_array = pca.fit_transform(X)
    
    # 3. Create principal component column names
    component_names = [f'PC{i+1}' for i in range(X_pca_array.shape[1])]
    
    # 4. Convert to DataFrame
    X_pca = pd.DataFrame(X_pca_array, columns=component_names)
    
    print(f"PCA reduced features from {X.shape[1]} to {X_pca.shape[1]} components.")
    
    return X_pca, pca, component_names


## Random Forest Model Training
Train a Random Forest classifier with optimized hyperparameters using RandomizedSearchCV.


In [31]:
def train_random_forest(X, y, class_weight=None):
   
    print("Training Random Forest model...")
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"Training set shape: {X_train.shape}")
    print(f"Testing set shape: {X_test.shape}")
    
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
    
    # Define hyperparameters for random search
    param_dist = {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [10, 20, 30, None],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__class_weight': [class_weight, 'balanced', 'balanced_subsample']
    }

    
    # Perform random search with cross-validation
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_dist,
        n_iter=10,
        cv=3,
        scoring='f1',
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit the model
    random_search.fit(X_train, y_train)
    
    # Get the best parameters
    best_params = random_search.best_params_
    print(f"Best parameters: {best_params}")
    
    # Get the best model
    best_model = random_search.best_estimator_
    
    # Evaluate the model
    evaluate_model(best_model, X_test, y_test)
    
    return best_model, X_test, y_test


## Model Evaluation
Evaluate the trained model using various metrics and visualizations.


In [33]:
def evaluate_model(model, X_test, y_test):
  
    print("Evaluating model...")
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Fire', 'Fire'], 
                yticklabels=['No Fire', 'Fire'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    
    # Create directory if it doesn't exist
    os.makedirs('../reports/figures', exist_ok=True)
    plt.savefig('../reports/figures/confusion_matrix.png')
    plt.close()
    
    # Feature importance
    if hasattr(model, 'named_steps') and hasattr(model.named_steps['classifier'], 'feature_importances_'):
        feature_importances = pd.DataFrame(
            model.named_steps['classifier'].feature_importances_,
            index=X_test.columns,
            columns=['Importance']
        ).sort_values('Importance', ascending=False)
        
        print("\nTop 10 Most Important Features:")
        print(feature_importances.head(10))
        
        plt.figure(figsize=(12, 8))
        feature_importances.head(15).plot(kind='barh')
        plt.title('Feature Importance')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.savefig('../reports/figures/feature_importance.png')
        plt.close()


## Save Model and Features
Save the trained model, PCA model, and feature names for later use in evaluation and visualization.


In [35]:
def save_model(model, feature_names, output_path, pca_model=None, component_names=None):
    
    print(f"Saving model to {output_path}...")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save the model
    joblib.dump(model, output_path)
    
    # Save PCA model if provided
    if pca_model is not None:
        pca_path = os.path.join(os.path.dirname(output_path), 'pca_model.pkl')
        joblib.dump(pca_model, pca_path)
        print(f"PCA model saved to {pca_path}")
    
    # Save feature names - use component names if PCA was applied
    feature_path = os.path.join(os.path.dirname(output_path), 'feature_names.txt')
    with open(feature_path, 'w') as f:
        if component_names is not None:
            for feature in component_names:
                f.write(f"{feature}\n")
        else:
            for feature in feature_names:
                f.write(f"{feature}\n")
    
    print(f"Model saved to {output_path}")
    print(f"Feature names saved to {feature_path}")


## Main Execution
Run the complete Random Forest model training pipeline.


In [37]:
def main():
    """Main function for training the Random Forest model."""
    start_time = time.time()
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Define file paths
    input_path = "../data/processed/engineered_la_fire_data.csv"
    output_path = "../models/la_fire_random_forest_model.pkl"
    
    # Load engineered data
    df = load_engineered_data(input_path)
    
    # Select features
    X, y, selected_features = select_features(df)
    
    # Handle class imbalance
    X_balanced, y_balanced, class_weight = handle_class_imbalance(X, y, method='class_weight')
    
    # Apply PCA with original explained variance
    X_pca, pca_model, component_names = apply_pca(X_balanced, explained_variance=0.95)

    
    # Train model
    model, X_test, y_test = train_random_forest(X_pca, y_balanced, class_weight)
    
    # Save model with PCA components
    save_model(model, selected_features, output_path, pca_model, component_names)
    
    # Save PCA model separately for use in evaluation
    pca_path = os.path.join(os.path.dirname(output_path), 'pca_model.pkl')
    joblib.dump(pca_model, pca_path)
    
    # Print execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution completed in {execution_time:.2f} seconds")
    print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    print("Random Forest model training completed successfully!")
    
    return model, X_test, y_test, selected_features, component_names


In [39]:
# Execute the Random Forest model training pipeline
model, X_test, y_test, selected_features, component_names = main()


Started at: 2025-04-26 14:22:12
Loading engineered data from ../data/processed/engineered_la_fire_data.csv...
Loaded data with shape: (224986, 49)

Loaded Engineered Data Head:
         date  Fire_Occurred STATION NAME  AWND  DAPR  MDPR  PGTM  PRCP  TAVG  \
0  2014-12-27              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
1  2014-12-28              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
2  2014-12-29              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
3  2014-12-30              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
4  2014-12-31              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   

   ...  season_Summer  season_Fall  TMAX_3D  TMIN_3D  PRCP_3D  PRCP_14D  \
0  ...          False         True      0.0      0.0      0.0       0.0   
1  ...          False         True      0.0      0.0      0.0       0.0   
2  ...          False         True      0.0      0.0      0.0       0.0   
3  ...          False         True  